# Untersuchung der Entwicklung von Verbraucherpreisen und Energiekosten 2023


## Übersicht


Verbraucherpreise sind während der Pandemie und insbesondere nach dem  Ukraine-Krieg stark gestiegen. Die prozentuale Veränderung gegenüber dem Vorjahr bildet dabei die Inflationsrate. Hauptsächlich werden die Energiekosten dafür verantwortlich gemacht. Nun wollen wir die Entwicklung des Verbraucherpreises und der Erzeugerpreise untersuchen.

**Mögliche Fragen:**
1. Wie stark sind die Preise gestiegen?
2. Wann sind die Preise gestiegen?
3. Welche Produkte sind besonders betroffen?
4. Ist der Anstieg an Verbraucherpreisen und Erzeugerpreisen proportional?

## Daten


Die Daten für die Preisentwicklung in Deutschland wird, mithilfe von Messgrößen, von dem Statistischen Bundesamt erhoben. Der **Verbraucherpreisindex (VPI)** misst die prozentuale Veränderung der Preise bestimmter Waren und Dienstleistungen für private Haushalte. Die Veränderung des VPI zum Vorjahr bildet die Inflationsrate. Der **Erzeugerpreisindex (EPI)**  

## Data Cleaning

### VPI

Die Verbraucherindex-Daten von 2018-2023 (Basisjahr=2020) finden wir auf folgender Seite des statistischen Bundesamtes: https://www.destatis.de/DE/Themen/Wirtschaft/Preise/Verbraucherpreisindex/Tabellen/Verbraucherpreise-12Kategorien.html 

Die monatlichen Indizes laden wir in **Power BI**.


<img src='Bilder/2-1-VPI_load.png' width="600"/>

Um sie zu transformieren öffnen wir sie in **PowerQuery**. Wir stellen fest, dass für die zweite Tabelle, aufgrund der zusätzlichen Notiz, falsche column header verwendet wurden. Wir übernehmen die erste Zeile als header und entfernen die zweite Zeile.


<img src='Bilder/2-2-VPI_Table1.png' width="1000"/>

Mithilfe der *fill* Funktion übernehmen wir die korrekten Jahreszahlen für die null-Werte. Wir erstellen mit *Add Column From Examples* eine neue Spalte in dem Format "Jahr Monat" und verknüpfen die Tabellen mit einem *INNER JOIN* auf der neuen Spalte.

<img src='Bilder/2-3-VPI_merge_date_columns.png' width="1000"/>

Wir korrigieren die Spaltennamen und Spalten-Datentypen und bringen mit der Funktion *Unpivot* die Tabelle von der wide form in die long form.


<img src='Bilder/2-4-VPI_unpivot.png' width="1000"/>

<img src='Bilder/2-5-EPI_transform.png' width="1000"/>

### EPI gewerbliche Produkte

Quelle: https://www.destatis.de/DE/Themen/Wirtschaft/Preise/Erzeugerpreisindex-gewerbliche-Produkte/Tabellen/Erzeugerpreise-GewProdukte-Ausgewaehlte-Indizes.html#241814

Dieses mal benutzen wir Python für das Data Cleaning. Dazu laden wir die entsprechende Tabelle als Excel Datei herunter. 

Wir laden die Daten in ein Pandas Dataframe

In [54]:
import pandas as pd

file_path = '1-EPI-Gewerbliche-Produkte-2.xlsx'
df = pd.read_excel(file_path)
df.head(8)

,"Producer price index for industrial products: Germany,\nmonths, product classification (GP2009 2-/3-/4-/5-/6-/\n9-digit codes/special items)",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73
0,Index of producer prices of industrial products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Germany,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Producer price index for industrial products (...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,GP2009 (2-digit codes): Industrial products,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,January,February,March,April,May,June,July,August,...,March,April,May,June,July,August,September,October,November,December
5,GP09-05,Kohle,97.3,97.3,98.9,98.4,98.4,98.4,98.3,98.3,...,141,141,143,148.6,...,...,...,...,...,...
6,GP09-06,Erdöl und Erdgas,97.6,97,96,99.1,101,105.2,109.5,108.8,...,252.8,217.5,198.6,171.6,...,...,...,...,...,...
7,GP09-08,"Steine und Erden, sonstige Bergbauerzeugnisse",103.9,104,104,104.3,105,104.9,105.2,105,...,152.1,152.5,153.1,153.6,...,...,...,...,...,...


Wir Erstellen eine Funktion um die Spalten-Namen zu bestimmen.

In [55]:
def column_names():
    # Name der ersten Spalte 
    columns = ['Gewerbliche Produkte']
    # Zeitraum in der Form: 'JJJJ-MM' 
    for year in range(2018, 2024):
        for month in range(1, 13):
            columns.append(f'{year}-{month:02d}')
    return columns

Wir löschen die Reihen mit falschen Daten und die Spalte mit den Codes des Bundesamtes. 

In [56]:
def clean_data(df):
    # Drop rows with missing data in column: 'Unnamed: 2'
    df = df.dropna(subset=['Unnamed: 1'])
    # Drop column: 'Producer price index for industrial products: Germany, months, product classification (GP2009 2-/3-/4-/5-/6-/ 9-digit codes/special items)'
    df = df.drop(columns=['Producer price index for industrial products: Germany,\nmonths, product classification (GP2009 2-/3-/4-/5-/6-/\n9-digit codes/special items)'])
    # Rename columns
    df.columns = column_names()
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,Gewerbliche Produkte,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,...,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12
5,Kohle,97.3,97.3,98.9,98.4,98.4,98.4,98.3,98.3,98.1,...,141,141,143,148.6,...,...,...,...,...,...
6,Erdöl und Erdgas,97.6,97,96,99.1,101,105.2,109.5,108.8,115,...,252.8,217.5,198.6,171.6,...,...,...,...,...,...
7,"Steine und Erden, sonstige Bergbauerzeugnisse",103.9,104,104,104.3,105,104.9,105.2,105,104.9,...,152.1,152.5,153.1,153.6,...,...,...,...,...,...
8,Nahrungsmittel und Futtermittel,104.5,104.3,104.8,104.8,105,105.3,105.4,105.8,106.1,...,145.9,146.3,145.9,146.1,...,...,...,...,...,...
9,Getränke,103.5,103.9,104.4,104.7,104.8,104.9,105.4,105.4,105.4,...,124.2,124.5,124.7,124.7,...,...,...,...,...,...
